In [1]:
from mcpyhive.hiveclient import PrestoClient
client = PrestoClient()

In [2]:
samples_data = client.read_sql("select * from test_mall.recommend_dssm_feature_samples")
client.close()

In [3]:
samples_data.shape

(2540611, 16)

In [4]:
samples_data.head()
# samples_data.to_csv(path_or_buf="dssm_0615.csv",index=False)

,company_id,company_id_label,area_id,sku_id,sku_id_label,clicked,sku_id_list,len_sku_id_list,class1_id_list,len_class1_id_list,class2_id_list,len_class2_id_list,class1_id,class2_id,code,len_code
0,11014924,68086,136,3161358,127906,0,"136239, 136239, 1873, 2602, 3491, 1873, 2602, ...",22,"40, 6, 39, 39, 39, 7, 7, 7, 3, 3, 9, 47, 47, 4...",22,"363, 16, 365, 365, 365, 52, 52, 52, 8, 50, 12,...",22,13,134,"110, 119",2
1,56119157,169244,53,67934,4743,0,"1065, 1065, 161196, 161196, 161196, 161196, 16...",192,"40, 40, 43, 43, 43, 43, 43, 43, 43, 43, 6, 6, ...",192,"339, 339, 375, 375, 375, 375, 375, 375, 375, 3...",192,1,1,104,1
2,56697533,172233,130,162900,9344,1,"15165, 21693, 21693, 23204, 23204, 11417, 1141...",52,"44, 8, 8, 8, 8, 8, 8, 12, 40, 40, 6, 6, 39, 39...",52,"356, 201, 201, 202, 202, 206, 206, 116, 363, 3...",52,1,1,52,1
3,13252021,74485,680,251280,15027,1,"105310, 105310, 121744, 121744, 121744, 121744...",386,"44, 44, 44, 44, 44, 44, 44, 44, 49, 49, 49, 49...",386,"381, 381, 356, 356, 356, 356, 385, 385, 438, 4...",386,3,66,104,1
4,34975715,126251,396,2429994,85271,1,"10624, 10624, 10624, 10624, 5250, 5250, 1632, ...",20,"40, 40, 6, 6, 39, 39, 39, 39, 39, 39, 7, 7, 7,...",20,"363, 363, 16, 16, 365, 365, 365, 365, 365, 365...",20,39,369,116,1


In [5]:
# 1. 数据处理
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#（1.2）打乱数据集
samples_data = shuffle(samples_data)
X = samples_data[["company_id_label", "area_id", "sku_id_label", "sku_id_list", "len_sku_id_list",  "class1_id_list", "len_class1_id_list", "class2_id_list", "len_class2_id_list", "class1_id", "class2_id", "code", "len_code"]]
y = samples_data["clicked"]

In [6]:
samples_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2540611 entries, 706555 to 982917
Data columns (total 16 columns):
company_id            int64
company_id_label      int64
area_id               int64
sku_id                int64
sku_id_label          int64
clicked               int64
sku_id_list           object
len_sku_id_list       int64
class1_id_list        object
len_class1_id_list    int64
class2_id_list        object
len_class2_id_list    int64
class1_id             int64
class2_id             int64
code                  object
len_code              int64
dtypes: int64(12), object(4)
memory usage: 329.5+ MB


In [7]:
print(samples_data.isnull().any()) #用来判断某列是否有缺失值

company_id            False
company_id_label      False
area_id               False
sku_id                False
sku_id_label          False
clicked               False
sku_id_list           False
len_sku_id_list       False
class1_id_list        False
len_class1_id_list    False
class2_id_list        False
len_class2_id_list    False
class1_id             False
class2_id             False
code                  False
len_code              False
dtype: bool


In [8]:
def modify_list(df, column, SEQ_LEN):
    big_list = [[int(i) for i in l.split(', ')] for l in df[column]]
    for i in range(len(big_list)):
        if len(big_list[i]) > SEQ_LEN:
            big_list[i] = big_list[i][0:SEQ_LEN]
        if len(big_list[i]) < SEQ_LEN:
            big_list[i] = big_list[i] + [0 for j in range(SEQ_LEN-len(big_list[i]))]
    return np.array(big_list)

In [9]:
#（1.3）转换数据存储格式
SEQ_LEN = 50
train_model_input = {
                    "company_id_label": np.array(X["company_id_label"]), \
                     "area_id": np.array(X["area_id"]), \
                    "code": modify_list(X, "code",SEQ_LEN = 6), \
                     "len_code": np.array(X["len_code"]), \
                    "sku_id_list": modify_list(X, "sku_id_list",SEQ_LEN = 50), \
                     "len_sku_id_list": np.array(X["len_sku_id_list"]), \
                     "class2_id_list": modify_list(X, "class2_id_list",SEQ_LEN = 50), \
                     "len_class2_id_list": np.array(X["len_class2_id_list"]), \
    
                     "sku_id_label": np.array(X["sku_id_label"]), \
                     "class1_id": np.array(X["class1_id"]), \
                     "class2_id": np.array(X["class2_id"])
                    }

train_label = np.array(y)


In [10]:
# 2. 构建模型

# （2.1）统计每个离散特征的词频量，构造特征参数
from deepctr.inputs import SparseFeat, VarLenSparseFeat
from tensorflow.python.keras.models import Model

embedding_dim = 32
SEQ_LEN = 50
user_feature_columns = [
                        SparseFeat('company_id_label', max(samples_data["company_id_label"])+1, embedding_dim),
                        SparseFeat("area_id", max(samples_data["area_id"])+1, embedding_dim),
                        VarLenSparseFeat(SparseFeat('sku_id_list', max(samples_data["sku_id_label"])+1, embedding_dim,use_hash=True,
                                                    embedding_name="sku_id_label"), SEQ_LEN, 'mean', 'len_sku_id_list'),
                        VarLenSparseFeat(SparseFeat('class2_id_list', max(samples_data["class2_id"])+1, embedding_dim,use_hash=True,
                                                    embedding_name="class2_id"), SEQ_LEN, 'mean', 'len_class2_id_list'),
                        VarLenSparseFeat(SparseFeat('code', max(samples_data["company_id_label"])+1, embedding_dim,use_hash=True,
                                                    embedding_name="company_id_label"), 6, 'mean', 'len_code'),
                        ]

item_feature_columns = [
                        SparseFeat('sku_id_label', max(samples_data["sku_id_label"])+1, embedding_dim), 
                       SparseFeat('class1_id', max(samples_data["class1_id"])+1, embedding_dim),
                       SparseFeat('class2_id', max(samples_data["class2_id"])+1, embedding_dim)
                       ]

In [11]:
item_feature_columns

[SparseFeat(name='sku_id_label', vocabulary_size=205275, embedding_dim=32, use_hash=False, dtype='int32', embedding_name='sku_id_label', group_name='default_group'),
 SparseFeat(name='class1_id', vocabulary_size=52, embedding_dim=32, use_hash=False, dtype='int32', embedding_name='class1_id', group_name='default_group'),
 SparseFeat(name='class2_id', vocabulary_size=530, embedding_dim=32, use_hash=False, dtype='int32', embedding_name='class2_id', group_name='default_group')]

In [12]:
from itertools import chain
from deepctr.inputs import SparseFeat,VarLenSparseFeat,create_embedding_matrix,embedding_lookup,get_dense_input,varlen_embedding_lookup,get_varlen_pooling_list,mergeDict
#深度匹配 输入 input_from_feature_columns
def input_from_feature_columns(features, feature_columns, l2_reg, init_std, seed, prefix='', seq_mask_zero=True,
                               support_dense=True, support_group=False,embedding_matrix_dict=None):
    sparse_feature_columns = list(
        filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if feature_columns else []
    varlen_sparse_feature_columns = list(
        filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []
    if embedding_matrix_dict is  None:
        embedding_matrix_dict = create_embedding_matrix(feature_columns, l2_reg, init_std, seed, prefix=prefix,
                                                        seq_mask_zero=seq_mask_zero)

    group_sparse_embedding_dict = embedding_lookup(embedding_matrix_dict, features, sparse_feature_columns)
    dense_value_list = get_dense_input(features, feature_columns)
    if not support_dense and len(dense_value_list) > 0:
        raise ValueError("DenseFeat is not supported in dnn_feature_columns")

    sequence_embed_dict = varlen_embedding_lookup(embedding_matrix_dict, features, varlen_sparse_feature_columns)
    group_varlen_sparse_embedding_dict = get_varlen_pooling_list(sequence_embed_dict, features,
                                                                 varlen_sparse_feature_columns)
    group_embedding_dict = mergeDict(group_sparse_embedding_dict, group_varlen_sparse_embedding_dict)
    if not support_group:
        group_embedding_dict = list(chain.from_iterable(group_embedding_dict.values()))
    return group_embedding_dict, dense_value_list

In [13]:
import tensorflow as tf
from deepctr.layers.utils import reduce_max, reduce_mean, reduce_sum, concat_func, div, softmax
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Layer
#深度匹配 层 核心 Similarity
class Similarity(Layer):

    def __init__(self, gamma=1, axis=-1, type='cos', **kwargs):
        self.gamma = gamma
        self.axis = axis
        self.type = type
        super(Similarity, self).__init__(**kwargs)

    def build(self, input_shape):
        # Be sure to call this somewhere!
        super(Similarity, self).build(input_shape)

    def call(self, inputs, **kwargs):
        query, candidate = inputs
        if self.type == "cos":
            query_norm = tf.norm(query, axis=self.axis)
            candidate_norm = tf.norm(candidate, axis=self.axis)
        cosine_score = reduce_sum(tf.multiply(query, candidate), -1)
        if self.type == "cos":
            cosine_score = div(cosine_score, query_norm * candidate_norm + 1e-8)
        cosine_score = tf.clip_by_value(cosine_score, -1, 1.0) * self.gamma
        return cosine_score

    def compute_output_shape(self, input_shape):
        return (None, 1)

    def get_config(self, ):
        config = {'gamma': self.gamma, 'axis': self.axis, 'type': self.type}
        base_config = super(Similarity, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [14]:
class DNN(Layer):
    def __init__(self, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, seed=1024, **kwargs):
        self.hidden_units = hidden_units
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.seed = seed
        self.l2_reg = l2_reg
        self.use_bn = use_bn
        super(DNN, self).__init__(**kwargs)

    def build(self, input_shape):
        input_size = input_shape[-1]
        hidden_units = [int(input_size)] + list(self.hidden_units)
        self.kernels = [self.add_weight(name='kernel' + str(i),
                                        shape=(
                                            hidden_units[i], hidden_units[i + 1]),
                                        initializer=glorot_normal(
                                            seed=self.seed),
                                        regularizer=l2(self.l2_reg),
                                        trainable=True) for i in range(len(self.hidden_units))]
        self.bias = [self.add_weight(name='bias' + str(i),
                                     shape=(self.hidden_units[i],),
                                     initializer=Zeros(),
                                     trainable=True) for i in range(len(self.hidden_units))]
        if self.use_bn:
            self.bn_layers = [tf.keras.layers.BatchNormalization() for _ in range(len(self.hidden_units))]

        self.dropout_layers = [tf.keras.layers.Dropout(self.dropout_rate, seed=self.seed + i) for i in
                               range(len(self.hidden_units))]

        self.activation_layers = [activation_layer(self.activation) for _ in range(len(self.hidden_units))]

        super(DNN, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, training=None, **kwargs):

        deep_input = inputs

        for i in range(len(self.hidden_units)):
            fc = tf.nn.bias_add(tf.tensordot(
                deep_input, self.kernels[i], axes=(-1, 0)), self.bias[i])
            if self.use_bn:
                fc = self.bn_layers[i](fc, training=training)

            fc = self.activation_layers[i](fc)

            fc = self.dropout_layers[i](fc, training=training)
            deep_input = fc

        return deep_input

    def compute_output_shape(self, input_shape):
        if len(self.hidden_units) > 0:
            shape = input_shape[:-1] + (self.hidden_units[-1],)
        else:
            shape = input_shape

        return tuple(shape)

    def get_config(self, ):
        config = {'activation': self.activation, 'hidden_units': self.hidden_units,
                  'l2_reg': self.l2_reg, 'use_bn': self.use_bn, 'dropout_rate': self.dropout_rate, 'seed': self.seed}
        base_config = super(DNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class PredictionLayer(Layer):
    def __init__(self, task='binary', use_bias=True, **kwargs):
        if task not in ["binary", "multiclass", "regression"]:
            raise ValueError("task must be binary,multiclass or regression")
        self.task = task
        self.use_bias = use_bias
        super(PredictionLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        if self.use_bias:
            self.global_bias = self.add_weight(
                shape=(1,), initializer=Zeros(), name="global_bias")

        # Be sure to call this somewhere!
        super(PredictionLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        x = inputs
        if self.use_bias:
            x = tf.nn.bias_add(x, self.global_bias, data_format='NHWC')
        if self.task == "binary":
            x = tf.sigmoid(x)

        output = tf.reshape(x, (-1, 1))

        return output

    def compute_output_shape(self, input_shape):
        return (None, 1)

    def get_config(self, ):
        config = {'task': self.task, 'use_bias': self.use_bias}
        base_config = super(PredictionLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [15]:
# （2.2）构建模型
from layers_core import PredictionLayer, DNN #深度点击率预估 层 核心 PredictionLayer, DNN
from deepctr.inputs import build_input_features, combined_dnn_input, create_embedding_matrix
from tensorflow.python.keras.models import Model


def DSSM(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(64, 32),
         item_dnn_hidden_units=(64, 32),
         dnn_activation='tanh', dnn_use_bn=False,
         l2_reg_dnn=0.01, l2_reg_embedding=1e-6, dnn_dropout=0.01, init_std=0.0001, seed=1024, metric='cos'):

    embedding_matrix_dict = create_embedding_matrix(user_feature_columns + item_feature_columns, l2_reg_embedding,
                                                    init_std, seed,
                                                    seq_mask_zero=True)

    user_features = build_input_features(user_feature_columns)
    user_inputs_list = list(user_features.values())
    user_sparse_embedding_list, user_dense_value_list = input_from_feature_columns(user_features,
                                                                                   user_feature_columns,
                                                                                   l2_reg_embedding, init_std, seed,
                                                                                   embedding_matrix_dict=embedding_matrix_dict)
    user_dnn_input = combined_dnn_input(user_sparse_embedding_list, user_dense_value_list)

    item_features = build_input_features(item_feature_columns)
    item_inputs_list = list(item_features.values())
    item_sparse_embedding_list, item_dense_value_list = input_from_feature_columns(item_features,
                                                                                   item_feature_columns,
                                                                                   l2_reg_embedding, init_std, seed,
                                                                                   embedding_matrix_dict=embedding_matrix_dict)
    item_dnn_input = combined_dnn_input(item_sparse_embedding_list, item_dense_value_list)

    user_dnn_out = DNN(user_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                       dnn_use_bn, seed, )(user_dnn_input)

    item_dnn_out = DNN(item_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                       dnn_use_bn, seed)(item_dnn_input)

    score = Similarity(type=metric)([user_dnn_out, item_dnn_out])

    output = PredictionLayer("binary", False)(score)

    model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

    model.__setattr__("user_input", user_inputs_list)
    model.__setattr__("item_input", item_inputs_list)
    model.__setattr__("user_embedding", user_dnn_out)
    model.__setattr__("item_embedding", item_dnn_out)

    return model


In [ ]:
model = DSSM(user_feature_columns, item_feature_columns)
model.compile(optimizer='adagrad', loss="binary_crossentropy", metrics=['accuracy'])

history = model.fit(train_model_input, train_label,
                    batch_size=400, epochs=5, verbose=1, validation_split=0.15, )#384

Epoch 1/5
5399/5399 [==============================] - 698s 129ms/step - loss: 1.2793 - accuracy: 0.7215 - val_loss: 0.8394 - val_accuracy: 0.7554
Epoch 2/5
4992/5399 [==========================>...] - ETA: 51s - loss: 0.7091 - accuracy: 0.7988

In [17]:
import matplotlib.pyplot as plt
 
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
print(plt.show())

<Figure size 640x480 with 1 Axes>

None


In [55]:
user_df = samples_data[["company_id", "company_id_label", "area_id", "code", "len_code", "sku_id_list", "len_sku_id_list", "class2_id_list", "len_class2_id_list"]]
user_df = user_df.drop_duplicates()
user_df['counter'] =  range(len(user_df))
print(user_df.shape)
user_df.head()

(176445, 10)


,company_id,company_id_label,area_id,code,len_code,sku_id_list,len_sku_id_list,class2_id_list,len_class2_id_list,counter
706555,57411239,177248,581,103,1,"91401, 91401, 165706, 165706, 32402, 32402, 97...",64,"436, 436, 434, 434, 374, 374, 429, 429, 379, 3...",64,0
1253754,12663802,73158,511,86,1,"161235, 161235, 13842, 13842, 13842, 13842, 13...",26,"80, 413, 397, 397, 397, 397, 397, 397, 365, 36...",26,1
1701888,40308674,152231,10,104,1,"75331, 75331, 75331, 40248, 40248, 40248, 4024...",10,"206, 206, 206, 379, 379, 134, 134, 341, 341, 341",10,2
1100066,55664333,167264,376,"145, 102, 99, 97",4,"115954, 115954, 171090, 171090, 59722, 59722, ...",70,"450, 450, 328, 328, 330, 330, 367, 367, 367, 3...",70,3
163971,10435249,63724,536,124,1,"8532, 8532, 109499, 45989, 45989, 45990, 45990...",90,"429, 429, 379, 124, 124, 124, 124, 467, 467, 4...",90,4


In [44]:
sku_df =  samples_data[["sku_id", "sku_id_label", "class1_id", "class2_id"]]
sku_df = sku_df.drop_duplicates()
sku_df['counter'] = range(len(sku_df))
print(sku_df.shape)
sku_df.head()

(155908, 5)


,sku_id,sku_id_label,class1_id,class2_id,counter
706555,2570112,91371,3,68,0
1253754,110469,6743,45,397,1
1701888,2141346,75331,41,341,2
1100066,815707,26696,5,26,3
163971,219333,12823,3,51,4


In [20]:
user_model_input = {    
                    "company_id_label": np.array(user_df["company_id_label"]), \
                     "area_id": np.array(user_df["area_id"]), \
                    "code": modify_list(user_df,"code",SEQ_LEN = 6), \
                     "len_code": np.array(user_df["len_code"]), \
                    "sku_id_list": modify_list(user_df,"sku_id_list",SEQ_LEN = 50), \
                     "len_sku_id_list": np.array(user_df["len_sku_id_list"]), \
                     "class2_id_list": modify_list(user_df,"class2_id_list",SEQ_LEN = 50), \
                     "len_class2_id_list": np.array(user_df["len_class2_id_list"])
                    }
user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
user_embs = user_embedding_model.predict(user_model_input, batch_size=2 ** 12)
print(user_embs.shape)

(176445, 32)


In [57]:
user_embedding = pd.DataFrame(user_embs,dtype=np.float32)
user_embedding['counter'] = user_embedding.index
user_embedding = pd.merge(user_df,user_embedding,on='counter',how='left')
user_embedding = user_embedding.drop(["company_id_label", "area_id", 'code', 'len_code', 'sku_id_list', "len_sku_id_list", "class2_id_list", "len_class2_id_list"],axis=1)
print(user_embedding.shape)
user_embedding.head()

(176445, 34)


,company_id,counter,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
0,57411239,0,-0.000847,-0.000820,-0.000665,-0.001152,-0.000680,-0.000048,0.000748,0.000198,...,-0.000389,-0.000275,-0.000308,-0.000574,0.000076,-0.000741,0.000148,0.000208,0.000572,-0.000213
1,12663802,1,-0.000650,-0.000258,-0.000472,-0.000675,-0.000146,-0.000255,0.000696,0.000141,...,-0.000457,-0.000678,-0.000490,-0.000476,-0.000345,-0.000377,-0.000317,0.000260,0.000206,-0.000405
2,40308674,2,-0.000520,-0.000018,-0.000293,-0.000568,0.000212,-0.000655,0.000718,0.000201,...,-0.000410,-0.001014,-0.000765,-0.000286,-0.000786,-0.000215,-0.000417,0.000360,-0.000047,-0.000569
3,55664333,3,-0.000237,0.000391,-0.000105,-0.000019,0.000383,-0.000637,0.000702,0.000163,...,-0.000476,-0.001290,-0.000817,-0.000130,-0.001340,0.000022,-0.000909,0.000604,-0.000602,-0.000860
4,10435249,4,-0.000707,-0.000750,-0.000492,-0.000905,-0.000599,-0.000097,0.000429,0.000082,...,-0.000204,-0.000142,-0.000132,-0.000350,0.000451,-0.000570,0.000396,-0.000017,0.000542,0.000112


In [21]:
item_model_input = {
                    "sku_id_label": np.array(sku_df["sku_id_label"]), \
                     "class1_id": np.array(sku_df["class1_id"]), \
                     "class2_id": np.array(sku_df["class2_id"])
                    }
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)
item_embs = item_embedding_model.predict(item_model_input, batch_size=2 ** 12)
print(item_embs.shape)

(155908, 32)


In [53]:
sku_embedding = pd.DataFrame(item_embs,dtype=np.float32)
sku_embedding['counter'] = sku_embedding.index
sku_embedding = pd.merge(sku_df,sku_embedding,on='counter',how='left')
sku_embedding = sku_embedding.drop(["sku_id_label", "class1_id", 'class2_id', 'counter'],axis=1)
print(sku_embedding.shape)
sku_embedding.head()

(155908, 33)


,sku_id,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,2570112,-0.000966,-0.000708,-0.000833,-0.001399,-0.000397,-0.000151,0.000936,0.000200,0.001024,...,-0.000650,-0.000736,-0.000643,-0.000693,-0.000119,-0.000758,-0.000214,0.000441,0.000466,-0.000272
1,110469,0.000937,0.000080,0.000636,0.000972,-0.000266,0.000714,-0.001317,-0.000246,-0.001085,...,0.000909,0.001631,0.001291,0.000626,0.001220,0.000431,0.000978,-0.000850,0.000131,0.001052
2,2141346,0.000138,-0.000407,-0.000097,-0.000196,-0.000504,0.000077,-0.000136,-0.000124,0.000004,...,0.000256,0.000909,0.000667,0.000015,0.000619,-0.000181,0.000705,-0.000101,0.000447,0.000461
3,815707,-0.000420,-0.000404,-0.000347,-0.000587,-0.000398,-0.000112,0.000083,-0.000114,0.000316,...,0.000027,0.000463,0.000022,-0.000092,0.000400,-0.000464,0.000307,0.000164,0.000581,0.000010
4,219333,-0.001407,-0.001013,-0.000997,-0.001639,-0.000397,-0.000346,0.001444,0.000239,0.001273,...,-0.000950,-0.000935,-0.001092,-0.000686,-0.000179,-0.000935,-0.000133,0.000536,0.000552,-0.000517


In [22]:
item_model_input2 = {
                    "sku_id_label": np.array(sku_df["sku_id_label"][0:1]), \
                     "class1_id": np.array(sku_df["class1_id"][0:1]), \
                     "class2_id": np.array(sku_df["class2_id"][0:1])
                    }
item_model_input2

{'sku_id_label': array([91371]),
 'class1_id': array([3]),
 'class2_id': array([68])}

In [23]:
item_embs2 = item_embedding_model.predict(item_model_input2, batch_size=2 ** 12)
item_embs2

array([[-9.6613629e-04, -7.0819521e-04, -8.3304348e-04, -1.3992902e-03,
        -3.9718824e-04, -1.5097627e-04,  9.3636330e-04,  1.9981906e-04,
         1.0243345e-03,  2.8668199e-04, -2.1315253e-04,  3.2918979e-06,
        -3.5350979e-04,  7.8199466e-04,  3.5950838e-04, -4.8220163e-04,
         1.3970234e-03, -7.4037717e-04, -1.3848569e-04,  9.7146956e-05,
        -2.4939136e-04, -1.6005277e-03, -6.5047183e-04, -7.3648646e-04,
        -6.4333819e-04, -6.9262821e-04, -1.1850102e-04, -7.5801084e-04,
        -2.1381839e-04,  4.4076151e-04,  4.6586743e-04, -2.7173187e-04]],
      dtype=float32)

In [ ]:
from tensorflow.python.keras.models import save_model,load_model
save_model(user_embedding_model, 'user_embedding_model.h5')# save_model
save_model(item_embedding_model, 'item_embedding_model.h5')# save_model
save_model(model, 'model.h5')# save_model

In [ ]:
from deepmatch.layers import custom_objects
model = load_model('model.h5',custom_objects)# load_model、
user_embedding_model = load_model('user_embedding_model.h5',custom_objects)
item_embedding_model = load_model('item_embedding_model.h5',custom_objects)

In [ ]:
# import findspark
# findspark.init()

# from pyspark import SparkContext, SparkConf
# from pyspark.sql.session import SparkSession
# spark = SparkSession.builder.appName("test").master('yarn').enableHiveSupport().getOrCreate()

# data = spark.sql('select *  from dim.dim_city limit 10')
# data.show()